In [9]:
import casadi as cs
import numpy as np
from casadi import *

In [10]:
A = MX.sym('A',3,3)
b = MX.sym('b',3)
print(solve(A,b))

(A\b)


In [71]:

kp = cs.SX.sym('kp')
kd = cs.SX.sym('kd')
L = cs.SX.sym('L')

x0 = cs.SX.sym('x0')
dx0 = cs.SX.sym('dx0')
y0 = cs.SX.sym('y0')
dy0 = cs.SX.sym('dy0')
phix = cs.SX.sym('phix')
dphix = cs.SX.sym('dphix')
phiy = cs.SX.sym('phiy')
dphiy = cs.SX.sym('dphiy')

gx = cs.SX.sym('gx')
gy = cs.SX.sym('gy')

dzdt = cs.SX.sym('dzdt', 8,1)

uy = - L*cy/cx*(kd*dphix + kp*phix) - 2.0*L*sy / \
    cx*dphix*dphiy + 9.81*sx*sy*sy/cx

# Eq 51-54 in Ecc
dzdt[0] = dx0  # dx0
dzdt[1] = gx + (L*(kd*dphiy + kp*phiy) - L*cy*sy *
                dphix*dphix - uy*sx*sy)/cy  # ddx0
dzdt[2] = dy0  # dy0
dzdt[3] = gy - (L*cy*(kd*dphix + kp*phix) + 2.0*L*sy *
                dphix*dphiy - 9.81*sx*sy*sy)/cx  # ddy0
dzdt[4] = z[5]  # dphix
dzdt[5] = (-kd*dphix) - (kp*phix) - \
    (9.81/L*cy*sx) + (gy*cx/(cy*L))  # ddphix
dzdt[6] = z[7]  # dphiy
dzdt[7] = (-kd*dphiy) - (kp*phiy) - (9.81 / L * cx*sy) - \
    ((gx*cy + gy*sx*sy)/L)  # ddphiy

print(dzdt)

[dzdt_0, dzdt_1, dzdt_2, dzdt_3, dzdt_4, dzdt_5, dzdt_6, dzdt_7]
@1=cos(phiy), @2=sin(phiy), @3=9.81, @4=sin(phix), @5=cos(phix), @6=2, [dx0, (gx+((((L*((kd*dphiy)+(kp*phiy)))-((((L*@1)*@2)*dphix)*dphix))-(((((((@3*@4)*@2)*@2)/@5)-((((L*@1)/@5)*((kd*dphix)+(kp*phix)))+(((((@6*L)*@2)/@5)*dphix)*dphiy)))*@4)*@2))/@1)), dy0, (gy-(((((L*@1)*((kd*dphix)+(kp*phix)))+((((@6*L)*@2)*dphix)*dphiy))-(((@3*@4)*@2)*@2))/@5)), dphix, (((gy*@5)/(@1*L))-(((kd*dphix)+(kp*phix))+(((@3/L)*@1)*@4))), dphiy, (-((((kd*dphiy)+(kp*phiy))+(((@3/L)*@5)*@2))+(((gx*@1)+((gy*@4)*@2))/L)))]


In [24]:
cx = cs.SX.cos(phix)
sx = cs.SX.sin(phix)
cy = cs.SX.cos(phiy)
sy = cs.SX.sin(phiy)

In [27]:
uy = - L*cy/cx*(kd*dphix + kp*phix) - 2.0*L*sy / cx*dphix*dphiy + 9.81*sx*sy*sy/cx
uy

SX(@1=sin(phiy), @2=cos(phix), (((((9.81*sin(phix))*@1)*@1)/@2)-((((L*cos(phiy))/@2)*((kd*dphix)+(kp*phix)))+(((((2*L)*@1)/@2)*dphix)*dphiy))))

In [30]:
cs.gradient(uy, phix)

SX(@1=9.81, @2=sin(phiy), @3=cos(phix), @4=((L*cos(phiy))/@3), ((cos(phix)*(@1*(@2*(@2/@3))))-((kp*@4)+(sin(phix)*(((((((2*L)*@2)/@3)/@3)*(dphix*dphiy))+((@4/@3)*((kd*dphix)+(kp*phix))))-(((((@1*sin(phix))*@2)*@2)/@3)/@3))))))

In [32]:
x = SX.sym('x',2)
y = SX.sym('y')
f = Function('f',[x,y], [x,sin(y)*x])
print(f)

f:(i0[2],i1)->(o0[2],o1[2]) SXFunction


In [33]:
A = np.random.rand(5,2)
A

array([[ 0.27184141,  0.34638923],
       [ 0.71980985,  0.68700698],
       [ 0.21999568,  0.82453792],
       [ 0.42201876,  0.82847081],
       [ 0.75160127,  0.55323616]])

In [35]:
z = SX.sym('x',nz)
x = SX.sym('x',nx)
g0 = sin(x+z)
g1 = cos(x-z)
g = Function('g',[z,x],[g0,g1])
G = rootfinder('G','newton',g)
print(G)

NameError: name 'nz' is not defined

In [36]:
x = SX.sym('x'); z = SX.sym('z'); p = SX.sym('p')
dae = {'x':x, 'z':z, 'p':p, 'ode':z+p, 'alg':z*cos(z)-x}
F = integrator('F', 'idas', dae)
print(F)

F:(x0,p,z0,rx0[0],rp[0],rz0[0])->(xf,qf[0],zf,rxf[0],rqf[0],rzf[0]) IdasInterface


In [38]:
x = SX.sym('x'); y = SX.sym('y')
qp = {'x':vertcat(x,y), 'f':x**2+y**2, 'g':x+y-10}
S = qpsol('S', 'qpoases', qp)
print(S)

S:(x0[2],p[],lbx[2],ubx[2],lbg,ubg,lam_x0[2],lam_g0)->(x[2],f,g,lam_x[2],lam_g,lam_p[]) MXFunction


In [39]:
r = S(lbg=0)
x_opt = r['x']
print('x_opt: ', x_opt)



####################   qpOASES  --  QP NO.   1   #####################

    Iter   |    StepLength    |       Info       |   nFX   |   nAC    
 ----------+------------------+------------------+---------+--------- 
       0   |   1.866661e-07   |   ADD CON    0   |     1   |     1   
       1   |   8.333622e-10   |   REM BND    1   |     0   |     1   
       2   |   1.000000e+00   |    QP SOLVED     |     0   |     1   
('x_opt: ', DM([5, 5]))


In [40]:
x = MX.sym('x',2)
y = MX.sym('y')
f = Function('f',[x,y],\
      [x,sin(y)*x],\
      ['x','y'],['r','q'])
f.generate('gen.c')
print(open('gen.c','r').read())

/* This file was automatically generated by CasADi.
   The CasADi copyright holders make no ownership claim of its contents. */
#ifdef __cplusplus
extern "C" {
#endif

/* How to prefix internal symbols */
#ifdef CODEGEN_PREFIX
  #define NAMESPACE_CONCAT(NS, ID) _NAMESPACE_CONCAT(NS, ID)
  #define _NAMESPACE_CONCAT(NS, ID) NS ## ID
  #define CASADI_PREFIX(ID) NAMESPACE_CONCAT(CODEGEN_PREFIX, ID)
#else
  #define CASADI_PREFIX(ID) gen_ ## ID
#endif

#include <math.h>

#ifndef casadi_real
#define casadi_real double
#endif

#ifndef casadi_int
#define casadi_int long long int
#endif

/* Add prefix to internal symbols */
#define casadi_copy CASADI_PREFIX(copy)
#define casadi_f0 CASADI_PREFIX(f0)
#define casadi_s0 CASADI_PREFIX(s0)
#define casadi_s1 CASADI_PREFIX(s1)

/* Symbol visibility in DLLs */
#ifndef CASADI_SYMBOL_EXPORT
  #if defined(_WIN32) || defined(__WIN32__) || defined(__CYGWIN__)
    #if defined(STATIC_LINKED)
      #define CASADI_SYMBOL_EXPORT
    #else
      #define CASADI_SYMB

In [42]:
!gcc -fPIC -shared gen.c -o gen.so

In [47]:
f = cs.external('f', './gen.so')
print(f(3.14, 0.3))

(DM([3.14, 3.14]), DM([0.927933, 0.927933]))


In [57]:
f = Function('f',[x],[sin(x)])
g = Function('g',[x],[cos(x)])
C = CodeGenerator('gen2.c')
C.add(f)
C.add(g)
C.generate()

'gen2.c'

In [60]:
!gcc -fPIC -shared gen2.c -o gen2.so

In [61]:
f = external('f', './gen2.so')
print(f(3.14))

[0.00159265, 0.00159265]


In [64]:
f = Function('f',[x,y],\
      [x,sin(y)*x],\
      ['x','y'],['r','q'])
print(f)

f:(x[2],y)->(r[2],q[2]) MXFunction


In [72]:
cd = Function('cd', 
              [x0, dx0, y0, dy0, phix, dphix, phiy, dphiy],
             )

NotImplementedError: Wrong number or type of arguments for overloaded function 'new_Function'.
  Possible prototypes are:
    Function()
    Function(str)
    Function(Function)
    Function(str,[SX],[SX],dict)
    Function(str,[MX],[MX],dict)
    Function(str,dict:SX,[str],[str],dict)
    Function(str,dict:MX,[str],[str],dict)
    Function(str,[SX],[SX],[str],[str],dict)
    Function(str,[MX],[MX],[str],[str],dict)
  You have: '(str,[SX])'


In [89]:

kp = cs.SX.sym('kp')
kd = cs.SX.sym('kd')
L = cs.SX.sym('L')

x0 = cs.SX.sym('x0')
dx0 = cs.SX.sym('dx0')
y0 = cs.SX.sym('y0')
dy0 = cs.SX.sym('dy0')
phix = cs.SX.sym('phix')
dphix = cs.SX.sym('dphix')
phiy = cs.SX.sym('phiy')
dphiy = cs.SX.sym('dphiy')

z = vertcat(x0, dx0, y0, dy0, phix, dphix, phiy, dphiy)
k = vertcat(kp, kd)

gx = cs.SX.sym('gx')
gy = cs.SX.sym('gy')
g = vertcat(gx, gy)

dzdt = cs.SX.sym('dzdt', 8,1)

In [93]:
Q = np.diag([1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])  # Eq. 64 in Ecc

In [97]:
Q = cs.SX('Q', np.diag([1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]))


NotImplementedError: Wrong number or type of arguments for overloaded function 'new_SX'.
  Possible prototypes are:
    SX()
    SX(Sparsity)
    SX(float)
    SX([[float]])
    SX([float])
    SX([int])
    SX(IM)
    SX(DM)
    SX(SX)
    SX(int,int)
    SX(Sparsity,SX)
  You have: '(str,np.array(float))'


In [98]:
Q = cs.SX.sym('Q', 8, 8)

In [103]:
cs.qpsol?

In [101]:
Q(np.diag([1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]))

TypeError: 'SX' object is not callable

In [87]:

def continuous_dynamics(z, g, k, L):
    kp = k[0]
    kd = k[1]
    
    x0 = z[0]
    dx0 = z[1]
    y0 = z[2]
    dy0 = z[3]
    phix = z[4]
    dphix = z[5]
    phiy = z[6]
    dphiy = z[7]
    
    cx = cs.SX.cos(phix)
    sx = cs.SX.sin(phix)
    cy = cs.SX.cos(phiy)
    sy = cs.SX.sin(phiy)

    gx = g[0]
    gy = g[1]

    uy = - L*cy/cx*(kd*dphix + kp*phix) - 2.0*L*sy / \
        cx*dphix*dphiy + 9.81*sx*sy*sy/cx

    # Eq 51-54 in Ecc
    dzdt = np.zeros(8)
    dzdt[0] = dx0  # dx0
    dzdt[1] = gx + (L*(kd*dphiy + kp*phiy) - L*cy*sy *
                    dphix*dphix - uy*sx*sy)/cy  # ddx0
    dzdt[2] = dy0  # dy0
    dzdt[3] = gy - (L*cy*(kd*dphix + kp*phix) + 2.0*L*sy *
                    dphix*dphiy - 9.81*sx*sy*sy)/cx  # ddy0
    dzdt[4] = z[5]  # dphix
    dzdt[5] = (-kd*dphix) - (kp*phix) - \
        (9.81/L*cy*sx) + (gy*cx/(cy*L))  # ddphix
    dzdt[6] = z[7]  # dphiy
    dzdt[7] = (-kd*dphiy) - (kp*phiy) - (9.81 / L * cx*sy) - \
        ((gx*cy + gy*sx*sy)/L)  # ddphiy
    return dzdt

In [88]:
continuous_dynamics(z, g, k, L)

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan])